In [ ]:
import pandas as pd
from openpyxl import load_workbook, Workbook
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl.styles import numbers, PatternFill, Border, Side, Font, Alignment
import os
from datetime import datetime
import numpy as np
import math

from config_parameters import (
    ANALYSIS_TICKER_INTERVAL_FOLDER,
    SUMMARY_SIMULATIONS_FILE,
    ANALYSIS_STRATEGIES_FILE1,
    SIMULATION_DATA_DICT
)


thin_border = Border(
    left=Side(style='thin'),
    right=Side(style='thin'),
    top=Side(style='thin'),
    bottom=Side(style='thin')
)

# Define fill colors for positive and standout values
light_green = PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")
dark_green = PatternFill(start_color="006100", end_color="006100", fill_type="solid")


def analyse_one_type_strategy(start_date, end_date, intervals):
        


    sheet_output = os.path.join(SUMMARY_SIMULATIONS_FILE, f"{start_date}_to_{end_date}.xlsx")




    # Load summary dataframe
    summary_df = pd.read_excel(sheet_output)

    # Filter data by selected categories
    #summary_df = summary_df[summary_df['Category'].isin(categories)]

    # Unique tickers and intervals
    unique_tickers = summary_df['Ticker'].unique()
    unique_intervals = summary_df['Interval'].unique()

    # --- Grouping data ---
    grouped = summary_df.groupby(['Strategy', 'Interval']).agg(
        Count=('P&L after fees', 'count'),
        Sum_Deposit=('Deposit', 'sum'),
        Sum_PnL=('P&L after fees', 'sum'),
        Sum_Number=('Number of Trades', 'sum')
    ).reset_index()

    grouped['PnL_per_Deposit'] = grouped['Sum_PnL'] / grouped['Sum_Deposit']

    # Create pivot table for P&L after fees
    pivot_df = pd.pivot_table(
        summary_df,
        values='P&L after fees',
        index='Strategy',
        columns='Interval',
        aggfunc='sum',
        fill_value=0,
        margins=True,
        margins_name='Total'
    )

    # 1. Collect all positive profit values from the pivot table
    # Remove 'Total' column (horizontal)
    pivot_df_clean = pivot_df.drop(columns='Total', errors='ignore')

    # Remove 'Total' row (vertical)
    pivot_df_clean = pivot_df_clean.drop(index='Total', errors='ignore')

# Collect all positive values only (excluding zeros and NaNs)
    positive_values = pivot_df_clean[pivot_df_clean > 0].values.flatten()
    positive_values = [v for v in positive_values if isinstance(v, (int, float)) and not pd.isna(v)]

    # 2. Determine min and max profit values for normalization
    if positive_values:
        print(positive_values)
        v_min = min(positive_values)
        v_max = max(positive_values)
        print(v_max)
        def get_green_shade(value):
            """
            Returns a dynamic green color based on the value's relative magnitude.
            The higher the value, the darker the green.
            """
            if value> v_max:
                # If the value is greater than the max, set it to max for color calculation
                value = v_max   
            if value < v_min:
                # If the value is less than the min, set it to min for color calculation
                value = v_min
            if v_max == v_min:
                # All values are the same – use a default medium green
                intensity = 200
            else:
                # Normalize value to range [0, 1]
                normalized = (value - v_min) / (v_max - v_min)
                # Scale intensity to range [100, 255]
                intensity = int(50 + normalized * (255 - 50))
            
            # Convert intensity to hex and build the green color hex string
            hex_green = format(intensity, '02X')
            return PatternFill(start_color=f"00{hex_green}00", end_color=f"00{hex_green}00", fill_type="solid")

    else:
        # 3. Fallback: if no positive values exist, use default light green
        def get_green_shade(value):
            return PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")


    # Reorder columns based on defined interval list
    desired_order = intervals + ['Total'] if 'Total' in pivot_df.columns else intervals
    existing_columns = [col for col in desired_order if col in pivot_df.columns]
    pivot_df = pivot_df[existing_columns]

    # Prepare Excel Workbook
    sheet_name = f"{start_date} - {end_date}"


    # Save to Excel file
    excel_path = ANALYSIS_STRATEGIES_FILE1

    # === Sheet handling ===
    if os.path.exists(excel_path):
        wb_existing = load_workbook(excel_path)

        # Remove existing sheet with same name
        if sheet_name in wb_existing.sheetnames:
            std = wb_existing[sheet_name]
            wb_existing.remove(std)
        wb = wb_existing

    else:
        wb = Workbook()
    # Ensure the current sheet exists
    if sheet_name not in wb.sheetnames:
        wb.create_sheet(title=sheet_name)


    ws = wb[sheet_name]
    ws.title = sheet_name

    # Add headers
    header = f"Start date: {start_date} | End date: {end_date}"
    ws.append([header])

    deposit_value = summary_df['Deposit'].iloc[0]
    deposit_per_interval = deposit_value * len(unique_tickers)
    total_strategy_deposit = deposit_per_interval * len(unique_intervals)
    #Categories: {', '.join(categories)} |
    header = f" Ticker count: {len(unique_tickers)} | Simulation deposit: {deposit_value}   | Single strategy interval deposit: {deposit_per_interval}  | Strategy deposit: {total_strategy_deposit} "
    ws.append([header])

    # Add the header row
    ws.append(["P&L after fees summary"])

    # Merge cells across the width of the interval columns
    merge_end_col = 2 + len(intervals)  # assuming you start at column 1
    ws.merge_cells(start_row=ws.max_row, start_column=1, end_row=ws.max_row, end_column=merge_end_col)

    # Center the text in the merged cell
    cell = ws.cell(row=ws.max_row, column=1)
    cell.alignment = Alignment(horizontal="center", vertical="center")

    # Round and convert pivot to integers
    pivot_df_rounded = pivot_df.round(0).astype(int)
    pivot_data = pivot_df_rounded.reset_index()
    first_strategy = pivot_df_rounded.index[0]

    # Insert pivot table into sheet
    for i, row in enumerate(dataframe_to_rows(pivot_data, index=False, header=True)):
        ws.append(row)
        if i == 0:
            continue

        current_row = ws.max_row
        strategy_name = row[0]

        for j, cell in enumerate(ws[current_row][1:], start=1):  # Skip 'Strategy' column
            value = cell.value
            interval = pivot_df_rounded.columns[j - 1]
            cell.number_format = '#,##0'
            cell.border = thin_border
            if isinstance(value, (int, float)):
                if value > 0:
                    cell.fill = get_green_shade(value)
                    reference_value = pivot_df_rounded.loc[first_strategy, interval]
                    if strategy_name != first_strategy and value > reference_value:
                        cell.fill = get_green_shade(value)

    

    wb.save(excel_path)
    print(f"✅ Workbook successfully saved to: {excel_path}")


def analyse_two_type_strategy(start_date, end_date, intervals):
        


    sheet_output = os.path.join(SUMMARY_SIMULATIONS_FILE, f"{start_date}_to_{end_date}.xlsx")




    # Load summary dataframe
    summary_df = pd.read_excel(sheet_output)

    # Filter data by selected categories
    #summary_df = summary_df[summary_df['Category'].isin(categories)]

    # Unique tickers and intervals
    unique_tickers = summary_df['Ticker'].unique()
    unique_intervals = summary_df['Interval'].unique()

    # --- Grouping data ---
    grouped = summary_df.groupby(['Strategy', 'Interval']).agg(
        Count=('P&L after fees', 'count'),
        Sum_Deposit=('Deposit', 'sum'),
        Sum_PnL=('P&L after fees', 'sum'),
        Sum_Number=('Number of Trades', 'sum')
    ).reset_index()

    grouped['PnL_per_Deposit'] = grouped['Sum_PnL'] / grouped['Sum_Deposit']

    # Create pivot table for P&L after fees
    pivot_df = pd.pivot_table(
        summary_df,
        values='P&L after fees',
        index='Strategy',
        columns='Interval',
        aggfunc='sum',
        fill_value=0,
        margins=True,
        margins_name='Total'
    )

    # 1. Collect all positive profit values from the pivot table
    # Remove 'Total' column (horizontal)
    pivot_df_clean = pivot_df.drop(columns='Total', errors='ignore')

    # Remove 'Total' row (vertical)
    pivot_df_clean = pivot_df_clean.drop(index='Total', errors='ignore')

# Collect all positive values only (excluding zeros and NaNs)
    positive_values = pivot_df_clean[pivot_df_clean > 0].values.flatten()
    positive_values = [v for v in positive_values if isinstance(v, (int, float)) and not pd.isna(v)]

    # 2. Determine min and max profit values for normalization
    if positive_values:
        print(positive_values)
        v_min = min(positive_values)
        v_max = max(positive_values)
        print(v_max)
        def get_green_shade(value):
            """
            Returns a dynamic green color based on the value's relative magnitude.
            The higher the value, the darker the green.
            """
            if value> v_max:
                # If the value is greater than the max, set it to max for color calculation
                value = v_max   
            if value < v_min:
                # If the value is less than the min, set it to min for color calculation
                value = v_min
            if v_max == v_min:
                # All values are the same – use a default medium green
                intensity = 200
            else:
                # Normalize value to range [0, 1]
                normalized = (value - v_min) / (v_max - v_min)
                # Scale intensity to range [100, 255]
                intensity = int(50 + normalized * (255 - 50))
            
            # Convert intensity to hex and build the green color hex string
            hex_green = format(intensity, '02X')
            return PatternFill(start_color=f"00{hex_green}00", end_color=f"00{hex_green}00", fill_type="solid")

    else:
        # 3. Fallback: if no positive values exist, use default light green
        def get_green_shade(value):
            return PatternFill(start_color="C6EFCE", end_color="C6EFCE", fill_type="solid")


    # Reorder columns based on defined interval list
    desired_order = intervals + ['Total'] if 'Total' in pivot_df.columns else intervals
    existing_columns = [col for col in desired_order if col in pivot_df.columns]
    pivot_df = pivot_df[existing_columns]

    # Prepare Excel Workbook
    sheet_name = f"{start_date} - {end_date}"


    # Save to Excel file
    excel_path = "COMPLEX_ANALYSES.xlsx"

    # === Sheet handling ===
    if os.path.exists(excel_path):
        wb_existing = load_workbook(excel_path)

        # Remove existing sheet with same name
        if sheet_name in wb_existing.sheetnames:
            std = wb_existing[sheet_name]
            wb_existing.remove(std)
        wb = wb_existing

    else:
        wb = Workbook()
    # Ensure the current sheet exists
    if sheet_name not in wb.sheetnames:
        wb.create_sheet(title=sheet_name)


    ws = wb[sheet_name]
    ws.title = sheet_name

    # Add headers
    header = f"Start date: {start_date} | End date: {end_date}"
    ws.append([header])

    deposit_value = summary_df['Deposit'].iloc[0]
    deposit_per_interval = deposit_value * len(unique_tickers)
    total_strategy_deposit = deposit_per_interval * len(unique_intervals)
    #Categories: {', '.join(categories)} |
    header = f" Ticker count: {len(unique_tickers)} | Simulation deposit: {deposit_value}   | Single strategy interval deposit: {deposit_per_interval}  | Strategy deposit: {total_strategy_deposit} "
    ws.append([header])

    # Add the header row
    ws.append(["P&L after fees summary"])

    # Merge cells across the width of the interval columns
    merge_end_col = 2 + len(intervals)  # assuming you start at column 1
    ws.merge_cells(start_row=ws.max_row, start_column=1, end_row=ws.max_row, end_column=merge_end_col)

    # Center the text in the merged cell
    cell = ws.cell(row=ws.max_row, column=1)
    cell.alignment = Alignment(horizontal="center", vertical="center")

    # Round and convert pivot to integers
    pivot_df_rounded = pivot_df.round(0).astype(int)
    pivot_data = pivot_df_rounded.reset_index()
    first_strategy = pivot_df_rounded.index[0]

    # Insert pivot table into sheet
    for i, row in enumerate(dataframe_to_rows(pivot_data, index=False, header=True)):
        ws.append(row)
        if i == 0:
            continue

        current_row = ws.max_row
        strategy_name = row[0]

        for j, cell in enumerate(ws[current_row][1:], start=1):  # Skip 'Strategy' column
            value = cell.value
            interval = pivot_df_rounded.columns[j - 1]
            cell.number_format = '#,##0'
            cell.border = thin_border
            if isinstance(value, (int, float)):
                if value > 0:
                    cell.fill = get_green_shade(value)
                    reference_value = pivot_df_rounded.loc[first_strategy, interval]
                    if strategy_name != first_strategy and value > reference_value:
                        cell.fill = get_green_shade(value)

    

    wb.save(excel_path)
    print(f"✅ Workbook successfully saved to: {excel_path}")



#categories = ['CRYPTO']




start_date = "2021-11-01"
start_date = "2010-01-01"
end_date = "2025-03-01"
period_id = '3'


# start_date = "2010-01-01"
# end_date = "2025-03-01"

# Define used intervals
intervals = ['1w', '3d', '2d', '1d', '12h', '6h', '4h', '2h', '1h', '30m', '15m']

analyse_two_type_strategy(start_date, end_date, intervals)

[300.01000000000005, 97.40999999999997, 301.36, 5.000000000000028, 3049.73, 5599.11, 18736.91, 16062.54, 8257.43, 83991.77, 16636.96, 3211.4700000000003, 10044.31, 29476.989999999998, 29127.72, 52971.57, 41977.73, 36973.44, 49563.65, 30806.48, 35639.43, 42305.94, 258764.94, 64195.96, 59511.05, 7560.97, 94451.31, 83422.31999999999, 30870.18, 35846.08, 292184.54000000004, 56812.28, 394613.48, 56046.11, 34855.96, 214.74999999999994, 815.4899999999999, 2199.09, 209.84, 3303.8100000000004, 3784.82, 823.09, 1764.48, 4572.72, 4519.91, 5231.87, 4479.5599999999995, 5102.35, 2362.31, 5086.46, 7139.67, 9937.85, 21465.409999999996, 6329.17, 7629.679999999999, 6405.72, 286.82999999999987, 34.41999999999999]
394613.48
✅ Workbook successfully saved to: COMPLEX_ANALYSES.xlsx
